In [1]:
import numpy as np
import pandas as pd
import theano
import theanets # autoencoders
from sklearn.cross_validation import train_test_split
from scipy import misc
import math

In [2]:
def crop(img):
    top, bottom, left, right = 30, -20, 15, -15
    return img[top:bottom, left:right]

In [3]:
def distance(c1, c2):
    (r1,g1,b1) = c1
    (r2,g2,b2) = c2
    return math.sqrt((r1 - r2)**2 + (g1 - g2) ** 2 + (b1 - b2) **2)

In [4]:
hit_map = {(255, 255, 255) : 0, # white nothing
           (0, 0, 0) : 0,       # homerun. not defensible
           (255, 0, 0) : 1,     # linedrive, laser
           (0, 255, 0) : 2,     # groundball  
           (0, 0, 255) : 3,     # flyball
           (160, 32, 240) : 4}  # blooper

colors = hit_map.keys()
    
def norm_color(rgb):
    rgb_key = tuple(rgb)
    
    if rgb_key in colors:
        return hit_map[rgb_key]
    else:    
        sc = sorted(colors, key=lambda color: distance(color, rgb))
        return hit_map[sc[0]]

In [5]:
mb = misc.imread('charts/pros/Mookie_Betts.png', mode='RGB')

In [6]:
mb = misc.imresize(mb, size=15)
mb = crop(mb)
misc.imshow(mb)

In [7]:
mb.shape

(97, 117, 3)

In [8]:
mb = [[norm_color(rgb) for rgb in row] for row in mb]

In [9]:
mb = np.array(mb)

In [13]:
mb = mb.reshape((1, 11349))

## One Hidden Layer

Autoencoder with only one hidden layer, the dimensions in the data captured by the
autoencoder model approximate the results of Principal Component Analysis (PCA). However, an autoencoder behaves much differently if there is non-linearity involved. And this csae study is very much non-linear. The autoencoder will detect different latent factors that PCA will never be able to detect.

In [21]:
model = theanets.Autoencoder([11349, (15,'relu'), 11349])
nonlinear_pca_model = model.train([mb], algo='rmsprop', input_noise=0.1, hidden_l1=.001, sparsity=0.9, num_updates=1000)

valid: 1 of 1 mini-batches from (1, 11349)
train: 1 of 1 mini-batches from (1, 11349)
downhill: compiling evaluation function
downhill: compiling RMSProp optimizer
downhill: setting: rms_halflife = 14
downhill: setting: rms_regularizer = 1e-08
downhill: setting: patience = 5
downhill: setting: validate_every = 10
downhill: setting: min_improvement = 0
downhill: setting: max_gradient_norm = 0
downhill: setting: max_gradient_elem = 0
downhill: setting: learning_rate = TensorConstant{0.0001}
downhill: setting: momentum = 0
downhill: setting: nesterov = False
downhill: validation 0 loss=2.557591 err=2.556549 *
downhill: RMSProp 1 loss=2.557357 err=2.556308
downhill: RMSProp 2 loss=2.542543 err=2.541714
downhill: RMSProp 3 loss=2.520956 err=2.519272
downhill: RMSProp 4 loss=2.484411 err=2.481968
downhill: RMSProp 5 loss=2.432532 err=2.429215
downhill: RMSProp 6 loss=2.369237 err=2.365099
downhill: RMSProp 7 loss=2.297068 err=2.292148
downhill: RMSProp 8 loss=2.217244 err=2.211536
downhill: 

In [22]:
nonlinear_pca_features = model.encode(mb)
print(nonlinear_pca_features)

[[ 18.75076021  24.33413124   0.           9.96437853   0.          20.27353259
    0.           0.           0.          23.78506166   0.           0.
   19.70279295  21.91068239   0.        ]]


In [ ]:
nonlinear_pca_features.shape

## Deeper Jeeper Creepers

More hidden layers, where the "black box" mystique of deep learners come in play. How they get the deep features or what these features even mean, is at best, a deep dark mystery. After some devoted research we can make head and tails of it. Even make a more sophiscated deep learner to figure it out, but then again, we won't know how it got it conclusions. This is the "devil in the details" dilemma, they are always one step ahead of us.

In [23]:
model = theanets.Autoencoder([11349, (15,'relu'), (15,'relu'), 11349])
deep_model = model.train([mb], algo='rmsprop', input_noise=0.1, hidden_l1=.001, sparsity=0.9, num_updates=1000)

valid: 1 of 1 mini-batches from (1, 11349)
train: 1 of 1 mini-batches from (1, 11349)
downhill: compiling evaluation function
downhill: compiling RMSProp optimizer
downhill: setting: rms_halflife = 14
downhill: setting: rms_regularizer = 1e-08
downhill: setting: patience = 5
downhill: setting: validate_every = 10
downhill: setting: min_improvement = 0
downhill: setting: max_gradient_norm = 0
downhill: setting: max_gradient_elem = 0
downhill: setting: learning_rate = TensorConstant{0.0001}
downhill: setting: momentum = 0
downhill: setting: nesterov = False
downhill: validation 0 loss=2.553673 err=2.552936 *
downhill: RMSProp 1 loss=2.553669 err=2.552926
downhill: RMSProp 2 loss=2.548905 err=2.548353
downhill: RMSProp 3 loss=2.543690 err=2.543027
downhill: RMSProp 4 loss=2.535996 err=2.535156
downhill: RMSProp 5 loss=2.524643 err=2.523577
downhill: RMSProp 6 loss=2.509495 err=2.508167
downhill: RMSProp 7 loss=2.491266 err=2.489669
downhill: RMSProp 8 loss=2.469369 err=2.467472
downhill: 

In [24]:
deep_features = model.encode(mb)
print(deep_features)

[[  0.          11.45285084   9.53832632   8.36055339  15.63655696   0.
   21.2409013   14.80379405  10.06881927   0.           0.           0.
    7.49961896   4.50466957   0.        ]]
